<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/Cross_Tabs_OLAP_Cube_v2_PGDM_FORDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Generating Cross Tabs
Companion codes for [this slide deck](https://docs.google.com/presentation/d/1BTMkvls1VZFZf-d2MF6xj4NvpeHFNLHums5KG5Z7Ewc/edit?usp=sharing) <br>
For a more complicated version of crosstabs using Normalised Key Value Tables see this [notebook](https://github.com/Praxis-QR/RDWH/blob/c783cd84b5155577388e77b69f84bf8a17e051cd/Cross_Tabs_with_OLAP_Cube_Normalised_KeyValueTables.ipynb)

In [ ]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2023-08-20 08:48:17.348532+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


#Install MySQL

In [ ]:
# Some system parameters
#!cat /proc/cpuinfo
#!cat /proc/meminfo

In [ ]:
!apt-get update > /dev/null
!apt-get -y install mysql-server > /dev/null
!mysql --version
!/etc/init.d/mysql restart

mysql  Ver 8.0.34-0ubuntu0.22.04.1 for Linux on x86_64 ((Ubuntu))
 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [ ]:
!mysql -e 'create database praxisDB'
!mysql -e 'show databases'

+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| praxisDB           |
| sys                |
+--------------------+


# Pandas

In [ ]:
!apt install libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
import pandas as pd
import MySQLdb
from sqlalchemy import create_engine, text

In [ ]:
connect_string = 'mysql://root@localhost/praxisDB'
engine = create_engine(connect_string)
#
DBConn = engine.connect()
#
df_mysql = pd.read_sql(text('show databases'), con=DBConn)
print ('loaded dataframe from MySQL. records:', len(df_mysql))
DBConn.close()
df_mysql

loaded dataframe from MySQL. records: 5


,Database
0,information_schema
1,mysql
2,performance_schema
3,praxisDB
4,sys


In [ ]:
#To run any non-SELECT SQL command
def runcmd (DDL):
    DBConn = engine.connect()
    try:
        retcode = DBConn.execute(text(DDL))
        DBConn.commit()
    except :
        print('Error : return code = ',retcode)
    DBConn.close()

#To run any SELECT SQL command and return data in pandas dataframe
def runselect (query):
    DBConn = engine.connect()
    df_mysql = pd.read_sql(text(query), con=DBConn)
    DBConn.close()
    return df_mysql

#Create, Load  Fact Table

In [ ]:
cmd = 'Drop table if exists CarSalesFact'
runcmd(cmd)
# ----------------------
cmd = 'Create table CarSalesFact(Make varchar(10), Type varchar(10), Colour varchar(10), Value decimal(8,0), Units int)'
runCMD(cmd)
# ----------------------

cmd = "\
INSERT INTO CarSalesFact (Make, Type, Colour, Value, Units) VALUES \
('Honda','Sedan','Brown',5000,1),('Honda','SUV','Black',18000, 4), ('Honda','SUV','Red',7000,2), \
('Mazda','Compact','Red',5000,1),('Mazda','Sedan','Brown',4000,1),('Mazda','SUV','Black',4000,1),('Mazda','SUV','Red',3000,1),\
('Toyota','Compact','Red',7000,1),('Toyota','Sedan','Black',9000,2)\
"
runcmd(cmd)
# ----------------------
#runSELECT('select * from CarSalesFact')
runselect( 'select * from CarSalesFact')

,Make,Type,Colour,Value,Units
0,Honda,Sedan,Brown,5000.0,1
1,Honda,SUV,Black,18000.0,4
2,Honda,SUV,Red,7000.0,2
3,Mazda,Compact,Red,5000.0,1
4,Mazda,Sedan,Brown,4000.0,1
5,Mazda,SUV,Black,4000.0,1
6,Mazda,SUV,Red,3000.0,1
7,Toyota,Compact,Red,7000.0,1
8,Toyota,Sedan,Black,9000.0,2


#Cross Tabs with FACT = UNIT

In [ ]:
runselect(" \
  select Make, Type, Colour, Units  \
  from CarSalesFact \
")

,Make,Type,Colour,Units
0,Honda,Sedan,Brown,1
1,Honda,SUV,Black,4
2,Honda,SUV,Red,2
3,Mazda,Compact,Red,1
4,Mazda,Sedan,Brown,1
5,Mazda,SUV,Black,1
6,Mazda,SUV,Red,1
7,Toyota,Compact,Red,1
8,Toyota,Sedan,Black,2


##Level 1

In [ ]:
runselect(" \
  select Make, Type, Sum(Units) Count \
  from CarSalesFact \
  group by Make, Type \
")

,Make,Type,Count
0,Honda,Sedan,1.0
1,Honda,SUV,6.0
2,Mazda,Compact,1.0
3,Mazda,Sedan,1.0
4,Mazda,SUV,2.0
5,Toyota,Compact,1.0
6,Toyota,Sedan,2.0


## Level 2

In [ ]:
runselect( " \
  select Make, Type, Sum(Units) Count \
  from CarSalesFact \
  group by Make, Type  \
")

,Make,Type,Count
0,Honda,Sedan,1.0
1,Honda,SUV,6.0
2,Mazda,Compact,1.0
3,Mazda,Sedan,1.0
4,Mazda,SUV,2.0
5,Toyota,Compact,1.0
6,Toyota,Sedan,2.0


##Level 2 - Rollup

In [ ]:
runselect( " \
  select Make, Type, Sum(Units) Count \
  from CarSalesFact \
  group by Make, Type  \
  with ROLLUP; \
")

,Make,Type,Count
0,Honda,Sedan,1.0
1,Honda,SUV,6.0
2,Honda,None,7.0
3,Mazda,Compact,1.0
4,Mazda,Sedan,1.0
5,Mazda,SUV,2.0
6,Mazda,None,4.0
7,Toyota,Compact,1.0
8,Toyota,Sedan,2.0
9,Toyota,None,3.0


In [ ]:
# Cosmetic improvement on report
runselect( " \
  select IFNULL(Make,'-total1-') 'Make', IFNULL(Type,'-total2-') 'Type', Sum(Units) Count \
  from CarSalesFact \
  group by Make, Type  \
  with ROLLUP; \
")

,Make,Type,Count
0,Honda,Sedan,1.0
1,Honda,SUV,6.0
2,Honda,-total2-,7.0
3,Mazda,Compact,1.0
4,Mazda,Sedan,1.0
5,Mazda,SUV,2.0
6,Mazda,-total2-,4.0
7,Toyota,Compact,1.0
8,Toyota,Sedan,2.0
9,Toyota,-total2-,3.0


### Roll UP even more

In [ ]:
runselect( " \
  select Make, Sum(Units) Count \
  from CarSalesFact \
  group by Make  \
  with ROLLUP; \
")

,Make,Count
0,Honda,7.0
1,Mazda,4.0
2,Toyota,3.0
3,None,14.0


In [ ]:
runselect( " \
  select IFNULL(Make,'-total-') 'Make', Sum(Units) Count \
  from CarSalesFact \
  group by Make  \
  with ROLLUP; \
")

,Make,Count
0,Honda,7.0
1,Mazda,4.0
2,Toyota,3.0
3,-total-,14.0


In [ ]:
runselect( " \
  select IFNULL(Type,'-total-') 'Type', Sum(Units) Count \
  from CarSalesFact \
  group by Type  \
  with ROLLUP; \
")

,Type,Count
0,Compact,2.0
1,Sedan,4.0
2,SUV,8.0
3,-total-,14.0


In [ ]:
runselect(" \
  select Sum(Units) 'Total' \
  from CarSalesFact \
")

,Total
0,14.0


##Level 3  - THREE crosstabs

In [ ]:
#This is the temporary table that we will create
#
# ---------------------------------------------
#
runselect( "\
SELECT \
	Make,\
	SUM(IF(Type='SUV',Units,0))  'SUV',\
	SUM(IF(Type='Sedan',Units,0))  'Sedan',\
	SUM(IF(Type='Compact',Units,0))  'Compact'\
  FROM CarSalesFact\
  GROUP BY Make WITH ROLLUP\
")

,Make,SUV,Sedan,Compact
0,Honda,6.0,1.0,0.0
1,Mazda,2.0,1.0,1.0
2,Toyota,0.0,2.0,1.0
3,None,8.0,4.0,2.0


In [ ]:
# Dimensions are TYPE and MAKE
# ---------------------------------
#
runselect(  "\
with sums as \
    (SELECT \
        Make,\
        SUM(IF(Type='SUV',Units,0))  'SUV',\
        SUM(IF(Type='Sedan',Units,0))  'Sedan',\
        SUM(IF(Type='Compact',Units,0))  'Compact'\
    FROM CarSalesFact\
    GROUP BY Make WITH ROLLUP\
    ) \
SELECT \
IFNULL(Make,'Col_Totals') Dims↓→,\
sums.SUV, sums.Sedan, sums.Compact,\
sums.SUV + sums.Sedan + sums.Compact  Row_Totals \
from sums \
")

,Dims↓→,SUV,Sedan,Compact,Row_Totals
0,Honda,6.0,1.0,0.0,7.0
1,Mazda,2.0,1.0,1.0,4.0
2,Toyota,0.0,2.0,1.0,3.0
3,Col_Totals,8.0,4.0,2.0,14.0


In [ ]:
# Dimensions are COLOUR and MAKE
# ---------------------------------
#
runselect(  "\
with sums as \
    (SELECT \
        Make,\
        SUM(IF(Colour='Brown',Units,0))  'Brown',\
        SUM(IF(Colour='Black',Units,0))  'Black',\
        SUM(IF(Colour='Red',Units,0))  'Red'\
    FROM CarSalesFact\
    GROUP BY Make WITH ROLLUP\
    ) \
SELECT \
IFNULL(Make,'Col_Totals') Dims↓→,\
sums.Brown, sums.Black, sums.Red,\
sums.Brown + sums.Black + sums.Red  Row_Totals \
FROM \
   sums \
")

,Dims↓→,Brown,Black,Red,Row_Totals
0,Honda,1.0,4.0,2.0,7.0
1,Mazda,1.0,1.0,2.0,4.0
2,Toyota,0.0,2.0,1.0,3.0
3,Col_Totals,2.0,7.0,5.0,14.0


In [ ]:
# Dimensions are COLOUR and TYPE
# ---------------------------------
#
runselect( "\
with sums as \
 ( \
  SELECT \
	Type,\
	SUM(IF(Colour='Brown',Units,0))  'Brown',\
	SUM(IF(Colour='Black',Units,0))  'Black',\
	SUM(IF(Colour='Red',Units,0))  'Red'\
  FROM CarSalesFact\
  GROUP BY Type WITH ROLLUP\
) \
SELECT \
IFNULL(Type,'Totals') Dims↓→,\
sums.Brown, sums.Black, sums.Red,\
sums.Brown + sums.Black + sums.Red  Totals \
from sums\
")


,Dims↓→,Brown,Black,Red,Totals
0,Compact,0.0,0.0,2.0,2.0
1,Sedan,2.0,2.0,0.0,4.0
2,SUV,0.0,5.0,3.0,8.0
3,Totals,2.0,7.0,5.0,14.0


##Level 3 A - the UNION ALL / CUBE

In [ ]:
runselect(  "\
SELECT Make, Type, SUM(Units) \
FROM CarSalesFact \
GROUP BY Make, Type \
UNION ALL \
SELECT Make, 'ALL', SUM(Units) \
FROM CarSalesFact \
GROUP BY Make \
UNION ALL \
SELECT 'ALL', Type, SUM(Units) \
FROM CarSalesFact \
GROUP BY Type \
UNION ALL \
SELECT 'ALL', 'ALL', SUM(Units) \
FROM CarSalesFact\
")



,Make,Type,SUM(Units)
0,Honda,Sedan,1.0
1,Honda,SUV,6.0
2,Mazda,Compact,1.0
3,Mazda,Sedan,1.0
4,Mazda,SUV,2.0
5,Toyota,Compact,1.0
6,Toyota,Sedan,2.0
7,Honda,ALL,7.0
8,Mazda,ALL,4.0
9,Toyota,ALL,3.0


#Cross Tabs with FACT = VALUE

In [ ]:
runselect( "select * from CarSalesFact")

,Make,Type,Colour,Value,Units
0,Honda,Sedan,Brown,5000.0,1
1,Honda,SUV,Black,18000.0,4
2,Honda,SUV,Red,7000.0,2
3,Mazda,Compact,Red,5000.0,1
4,Mazda,Sedan,Brown,4000.0,1
5,Mazda,SUV,Black,4000.0,1
6,Mazda,SUV,Red,3000.0,1
7,Toyota,Compact,Red,7000.0,1
8,Toyota,Sedan,Black,9000.0,2


##Level 1 - Partial Cross Tab

In [ ]:
runselect( "\
select Make, Type, sum(Units) Count, sum(Value) Total \
from \
CarSalesFact \
group by Make, Type\
")

,Make,Type,Count,Total
0,Honda,Sedan,1.0,5000.0
1,Honda,SUV,6.0,25000.0
2,Mazda,Compact,1.0,5000.0
3,Mazda,Sedan,1.0,4000.0
4,Mazda,SUV,2.0,7000.0
5,Toyota,Compact,1.0,7000.0
6,Toyota,Sedan,2.0,9000.0


##Level 2 Roll UP

In [ ]:
runselect( "\
select Make, Type, sum(Units) Count, sum(Value) Total \
from \
CarSalesFact \
group by Make, Type with ROllUP\
")

,Make,Type,Count,Total
0,Honda,Sedan,1.0,5000.0
1,Honda,SUV,6.0,25000.0
2,Honda,None,7.0,30000.0
3,Mazda,Compact,1.0,5000.0
4,Mazda,Sedan,1.0,4000.0
5,Mazda,SUV,2.0,7000.0
6,Mazda,None,4.0,16000.0
7,Toyota,Compact,1.0,7000.0
8,Toyota,Sedan,2.0,9000.0
9,Toyota,None,3.0,16000.0


##Level 3 Full Cross Tab

In [ ]:
runselect("\
with sums as \
    (SELECT \
        Make,\
        SUM(IF(Type='SUV',Value,0))  'SUV',\
        SUM(IF(Type='Sedan',Value,0))  'Sedan',\
        SUM(IF(Type='Compact',Value,0))  'Compact'\
    FROM CarSalesFact\
    GROUP BY Make WITH ROLLUP\
    ) \
SELECT \
IFNULL(Make,'Totals') Dims↓→,\
sums.SUV, sums.Sedan, sums.Compact,\
sums.SUV + sums.Sedan + sums.Compact  Totals \
FROM  \
   sums \
")

,Dims↓→,SUV,Sedan,Compact,Totals
0,Honda,25000.0,5000.0,0.0,30000.0
1,Mazda,7000.0,4000.0,5000.0,16000.0
2,Toyota,0.0,9000.0,7000.0,16000.0
3,Totals,32000.0,18000.0,12000.0,62000.0


In [ ]:
runselect (" \
with sums as \
    (SELECT \
        Make,\
        SUM(IF(Colour='Brown',Value,0))  'Brown',\
        SUM(IF(Colour='Black',Value,0))  'Black',\
        SUM(IF(Colour='Red',Value,0))  'Red'\
    FROM CarSalesFact\
    GROUP BY Make WITH ROLLUP \
    ) \
SELECT \
IFNULL(Make,'Totals') DimsDims↓→,\
sums.Brown, sums.Black, sums.Red,\
sums.Brown + sums.Black + sums.Red  Totals \
FROM  \
   sums \
")

,DimsDims↓→,Brown,Black,Red,Totals
0,Honda,5000.0,18000.0,7000.0,30000.0
1,Mazda,4000.0,4000.0,8000.0,16000.0
2,Toyota,0.0,9000.0,7000.0,16000.0
3,Totals,9000.0,31000.0,22000.0,62000.0


In [ ]:
runselect (" \
with sums as \
    (SELECT \
        Type,\
        SUM(IF(Colour='Brown',Value,0))  'Brown',\
        SUM(IF(Colour='Black',Value,0))  'Black',\
        SUM(IF(Colour='Red',Value,0))  'Red'\
    FROM CarSalesFact\
    GROUP BY Type WITH ROLLUP \
    ) \
SELECT \
IFNULL(Type,'Totals') DimsDims↓→,\
sums.Brown, sums.Black, sums.Red,\
sums.Brown + sums.Black + sums.Red  Totals \
FROM  \
   sums \
")

,DimsDims↓→,Brown,Black,Red,Totals
0,Compact,0.0,0.0,12000.0,12000.0
1,Sedan,9000.0,9000.0,0.0,18000.0
2,SUV,0.0,22000.0,10000.0,32000.0
3,Totals,9000.0,31000.0,22000.0,62000.0


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)